<a href="https://colab.research.google.com/github/elcintimurcakmak/A-B-Testing/blob/main/eniac_A_B_testing_with_four_button_versions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

elcintimurcakmak_return_rate_path = kagglehub.dataset_download('elcintimurcakmak/return-rate')
elcintimurcakmak_drop_off_path = kagglehub.dataset_download('elcintimurcakmak/drop-off')

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

# Hypothesis testing: Chi-Square Test within the Eniac case study

In this notebook we perform a chi-square test with the data from the Eniac case study, applying a post-hoc correction to perform pairwise tests and find the true winner.

**State the Null Hypothesis and the Alternative Hypothesis.**

$H_0$: All four versions perform the same / have the CTR       
$H_A$: One or more version have a significantly different CTR

**Select an appropriate significance level alpha ($\alpha$).**

It was decided that a relatively high alpha was acceptable in this case

In [ ]:
alpha = 0.1

In [ ]:
def open_file(url):
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    return pd.read_csv(path)

links = {
    "eniac_a": "https://drive.google.com/file/d/1r3ZKHXmL8BW7iuxN0YwBN5j9MgKEmkfp/view?usp=sharing",
    "eniac_b": "https://drive.google.com/file/d/19GdASTvjdkyT9nR6dEg43idtBD0FZ6y7/view?usp=sharing",
    "eniac_c": "https://drive.google.com/file/d/15FnpEwi0N8KxQE9dYl_YPnMwp56QsgnX/view?usp=sharing",
    "eniac_d": "https://drive.google.com/file/d/11qC4UvgsSghsM81M1E5QO6Y_eAq_fePz/view?usp=sharing",
}

df_a = open_file(links["eniac_a"])
df_b = open_file(links["eniac_b"])
df_c = open_file(links["eniac_c"])
df_d = open_file(links["eniac_d"])

In [ ]:
df_c.head()

In [ ]:
observed = pd.DataFrame(columns=["A", "B", "C", "D"], index=["Click", "Visits"], dtype="int")

# relevent webpage elements have been located on line 21
observed.loc["Click", "A"] = df_a.loc[21, "No. clicks"]
observed.loc["Click", "B"] = df_b.loc[21, "No. clicks"]
observed.loc["Click", "C"] = df_c.loc[21, "No. clicks"]
observed.loc["Click", "D"] = df_d.loc[21, "No. clicks"]

observed

In [ ]:
# with pandas
df_a["Snapshot information"].str.extract(r"(\d+ visits)").loc[1,0].split()[0]

In [ ]:
# with Python's re module
import re
re.search(r"\d+ visits", df_a.loc[1, "Snapshot information"]).group(0).split()[0]

In [ ]:
observed.loc["Visits", "A"] = int(df_a["Snapshot information"].str.extract(r"(\d+ visits)").loc[1, 0].split()[0])
observed.loc["Visits", "B"] = int(df_b["Snapshot information"].str.extract(r"(\d+ visits)").loc[1, 0].split()[0])
observed.loc["Visits", "C"] = int(df_c["Snapshot information"].str.extract(r"(\d+ visits)").loc[1, 0].split()[0])
observed.loc["Visits", "D"] = int(df_d["Snapshot information"].str.extract(r"(\d+ visits)").loc[1, 0].split()[0])

observed

In [ ]:
observed.loc["CTR"] = observed.loc["Click"] / observed.loc["Visits"] * 100
observed

In [ ]:
observed.loc["Not Click"] = observed.loc["Visits"] - observed.loc["Click"]
observed

**Calculate the test result**

In [ ]:
observed.loc[["Click", "Not Click"]]
results = stats.chi2_contingency(observed.loc[["Click", "Not Click"]])

In [ ]:
results.pvalue

**Interpret the test result**

In [ ]:
if results.pvalue < alpha:
    print("We reject the null hypothesis")
else:
    print("We do not reject the null hypothesis")

**How do we decide who's the winner?**

**Use a run-off**

In [ ]:
import itertools

list(itertools.combinations("ABCD", 2))

In [ ]:
import itertools

# list(itertools.combinations("ABCD", 2))

for combo in itertools.combinations("ABCD", 2):
    #print(stats.chi2_contingency(observed.loc[["Click", "Not Click"], combo]))
    if stats.chi2_contingency(observed.loc[["Click", "Not Click"], combo]).pvalue < (alpha / len(list(itertools.combinations("ABCD", 2)))):
        print(f"Combo {combo} shows significance")

In [ ]:
observed

In [ ]:
from IPython.display import Image
Image(filename='/kaggle/input/drop-off/Bildschirmfoto 2026-02-10 um 18.13.12.png')

In [ ]:
Image(filename='/kaggle/input/return-rate/Bildschirmfoto 2026-02-10 um 18.13.20.png')

With no significant difference in CTR between versions A and C, and conflicting indications from drop-off and homepage return rates, we recommend not changing the SHOP NOW button on the homepage.